In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [1]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls /content/drive/

mkdir: cannot create directory ‘drive’: File exists
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
'Colab Notebooks'  'logs (da070523)'  'model (88ddb4be)'


In [0]:
import sys
import os
import urllib.request
import tarfile
import zipfile


def _print_download_progress(count, block_size, total_size):
    """
    Function used for printing the download progress.
    Used as a call-back function in maybe_download_and_extract().
    """

    # Percentage completion.
    pct_complete = float(count * block_size) / total_size

    # Limit it because rounding errors may cause it to exceed 100%.
    pct_complete = min(1.0, pct_complete)

    # Status-message. Note the \r which means the line should overwrite itself.
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()


def _maybe_download_and_extract(url, download_dir):
    """
    Download and extract the data if it doesn't already exist.
    Assumes the url is a tar-ball file.
    :param url:
        Internet URL for the tar-file to download.
        Example: "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    :param download_dir:
        Directory where the downloaded file is saved.
        Example: "data/CIFAR-10/"
    :return:
        Nothing.
    """

    # Filename for saving the file downloaded from the internet.
    # Use the filename from the URL and add it to the download_dir.
    filename = url.split('/')[-1]
    file_path = os.path.join(download_dir, filename)

    # Check if the file already exists.
    # If it exists then we assume it has also been extracted,
    # otherwise we need to download and extract it now.
    if not os.path.exists(file_path):
        # Check if the download directory exists, otherwise create it.
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        # Download the file from the internet.
        file_path, _ = urllib.request.urlretrieve(url=url,
                                                  filename=file_path,
                                                  reporthook=_print_download_progress)

        print()
        print("Download finished. Extracting files.")

        if file_path.endswith(".zip"):
            # Unpack the zip-file.
            zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
        elif file_path.endswith((".tar.gz", ".tgz")):
            # Unpack the tar-ball.
            tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)

        print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")

In [0]:
import numpy as np


def one_hot_encoded(class_numbers, num_classes=None):
    """
    Generate the One-Hot encoded class-labels from an array of integers.
    For example, if class_number=2 and num_classes=4 then
    the one-hot encoded label is the float array: [0. 0. 1. 0.]
    :param class_numbers:
        Array of integers with class-numbers.
        Assume the integers are from zero to num_classes-1 inclusive.
    :param num_classes:
        Number of classes. If None then use max(class_numbers)+1.
    :return:
        2-dim array of shape: [len(class_numbers), num_classes]
    """

    # Find the number of classes if None is provided.
    # Assumes the lowest class-number is zero.
    if num_classes is None:
        num_classes = np.max(class_numbers) + 1

    return np.eye(num_classes, dtype=float)[class_numbers]

In [0]:
import numpy as np
import pickle
import os

########################################################################

# Directory where you want to download and save the data-set.
# Set this before you start calling any of the functions below.
data_path = "data/CIFAR-10/"

# URL for the data-set on the internet.
data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

########################################################################
# Various constants for the size of the images.
# Use these constants in your own program.

# Width and height of each image.
img_size = 32

# Number of channels in each image, 3 channels: Red, Green, Blue.
num_channels = 3

# Length of an image when flattened to a 1-dim array.
img_size_flat = img_size * img_size * num_channels

# Number of classes.
num_classes = 10

########################################################################
# Various constants used to allocate arrays of the correct size.

# Number of files for the training-set.
_num_files_train = 5

# Number of images for each batch-file in the training-set.
_images_per_file = 10000

# Total number of images in the training-set.
# This is used to pre-allocate arrays for efficiency.
_num_images_train = _num_files_train * _images_per_file

########################################################################
# Private functions for downloading, unpacking and loading data-files.


def _get_file_path(filename=""):
    """
    Return the full path of a data-file for the data-set.
    If filename=="" then return the directory of the files.
    """

    return os.path.join(data_path, "cifar-10-batches-py/", filename)


def _unpickle(filename):
    """
    Unpickle the given file and return the data.
    Note that the appropriate dir-name is prepended the filename.
    """

    # Create full path for the file.
    file_path = _get_file_path(filename)

    print("Loading data: " + file_path)

    with open(file_path, mode='rb') as file:
        # In Python 3.X it is important to set the encoding,
        # otherwise an exception is raised here.
        data = pickle.load(file, encoding='bytes')

    return data


def _convert_images(raw):
    """
    Convert images from the CIFAR-10 format and
    return a 4-dim array with shape: [image_number, height, width, channel]
    where the pixels are floats between 0.0 and 1.0.
    """

    # Convert the raw images from the data-files to floating-points.
    raw_float = np.array(raw, dtype=float) / 255.0

    # Reshape the array to 4-dimensions.
    images = raw_float.reshape([-1, num_channels, img_size, img_size])

    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])

    return images


def _load_data(filename):
    """
    Load a pickled data-file from the CIFAR-10 data-set
    and return the converted images (see above) and the class-number
    for each image.
    """

    # Load the pickled data-file.
    data = _unpickle(filename)

    # Get the raw images.
    raw_images = data[b'data']

    # Get the class-numbers for each image. Convert to numpy-array.
    cls = np.array(data[b'labels'])

    # Convert the images.
    images = _convert_images(raw_images)

    return images, cls


########################################################################
# Public functions that you may call to download the data-set from
# the internet and load the data into memory.


def maybe_download_and_extract():
    """
    Download and extract the CIFAR-10 data-set if it doesn't already exist
    in data_path (set this variable first to the desired path).
    """

    _maybe_download_and_extract(url=data_url, download_dir=data_path)


def load_class_names():
    """
    Load the names for the classes in the CIFAR-10 data-set.
    Returns a list with the names. Example: names[3] is the name
    associated with class-number 3.
    """

    # Load the class-names from the pickled file.
    raw = _unpickle(filename="batches.meta")[b'label_names']

    # Convert from binary strings.
    names = [x.decode('utf-8') for x in raw]

    return names


def load_training_data():
    """
    Load all the training-data for the CIFAR-10 data-set.
    The data-set is split into 5 data-files which are merged here.
    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    # Pre-allocate the arrays for the images and class-numbers for efficiency.
    images = np.zeros(shape=[_num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[_num_images_train], dtype=int)

    # Begin-index for the current batch.
    begin = 0

    # For each data-file.
    for i in range(_num_files_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = _load_data(filename="data_batch_" + str(i + 1))

        # Number of images in this batch.
        num_images = len(images_batch)

        # End-index for the current batch.
        end = begin + num_images

        # Store the images into the array.
        images[begin:end, :] = images_batch

        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch

        # The begin-index for the next batch is the current end-index.
        begin = end

    return images, one_hot_encoded(class_numbers=cls, num_classes=num_classes)


def load_test_data():
    """
    Load all the test-data for the CIFAR-10 data-set.
    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    images, cls = _load_data(filename="test_batch")

    return images, one_hot_encoded(class_numbers=cls, num_classes=num_classes)

In [0]:
import random
import numpy as np


def _random_crop(batch, crop_shape, padding=None):
        oshape = np.shape(batch[0])
        
        if padding:
            oshape = (oshape[0] + 2 * padding, oshape[1] + 2 * padding)
        new_batch = []
        npad = ((padding, padding), (padding, padding), (0, 0))
        for i in range(len(batch)):
            new_batch.append(batch[i])
            if padding:
                new_batch[i] = np.lib.pad(batch[i], pad_width=npad, mode="constant", constant_values=0)
            nh = random.randint(0, oshape[0] - crop_shape[0])
            nw = random.randint(0, oshape[1] - crop_shape[1])
            new_batch[i] = new_batch[i][nh:nh + crop_shape[0], nw:nw + crop_shape[1]]
        return new_batch


def _random_flip_leftright(batch):
        for i in range(len(batch)):
            if bool(random.getrandbits(1)):
                batch[i] = np.fliplr(batch[i])
        return batch


def data_augmentation(batch, crop_shape, size):
    batch = _random_flip_leftright(batch)
    batch = _random_crop(batch, [size,size], crop_shape)
    return batch

In [6]:
import tensorflow as tf

epochs = 200
batch_size = 100
num_filters = 32
window_size = 3
pooling_ratio = [1, 1.41, 1.41, 1]
learning_rate = 0.001
crop_shape = 4
log_save_path = "/content/drive/logs/"
model_save_path = "/content/drive/model/"

maybe_download_and_extract()
X_train, y_train = load_training_data()
X_test, y_test = load_test_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# z-score
X_train = (X_train - np.mean(X_train, axis=(0, 1, 2, 3))) / np.std(X_train,axis=(0,1,2,3))
X_test = (X_test - np.mean(X_test, axis=(0, 1, 2, 3))) / np.std(X_test,axis=(0,1,2,3))

x = tf.placeholder(tf.float32, [None, img_size, img_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_classes])

# 2D convolution layer
W_conv1 = tf.get_variable("W_conv1", [window_size, window_size, num_channels, num_filters])
b_conv1 = tf.get_variable("b_conv1", [num_filters])
h_conv1 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding="SAME"), b_conv1)))

# 2D convolution layer
W_conv2 = tf.get_variable("W_conv2", [window_size, window_size, num_filters, num_filters])
b_conv2 = tf.get_variable("b_conv2", [num_filters])
h_conv2 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(h_conv1, W_conv2, strides=[1, 1, 1, 1], padding="SAME"), b_conv2)))

# fractional max pooling layer
h_pool1 = tf.nn.fractional_max_pool(h_conv2, pooling_ratio, pseudo_random=True, overlapping=True)[0]

# dropout layer
dropout1 = tf.nn.dropout(h_pool1, 0.8)

# 2D convolution layer
W_conv3 = tf.get_variable("W_conv3", [window_size, window_size, num_filters, num_filters * 2])
b_conv3 = tf.get_variable("b_conv3", [num_filters * 2])
h_conv3 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(dropout1, W_conv3, strides=[1, 1, 1, 1], padding="SAME"), b_conv3)))

# 2D convolution layer
W_conv4 = tf.get_variable("W_conv4", [window_size, window_size, num_filters * 2, num_filters * 2])
b_conv4 = tf.get_variable("b_conv4", [num_filters * 2])
h_conv4 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding="SAME"), b_conv4)))

# fractional max pooling layer
h_pool2 = tf.nn.fractional_max_pool(h_conv4, pooling_ratio, pseudo_random=True, overlapping=True)[0]

# dropout layer
dropout2 = tf.nn.dropout(h_pool2, 0.8)

# 2D convolution layer
W_conv5 = tf.get_variable("W_conv5", [window_size, window_size, num_filters * 2, num_filters * 3])
b_conv5 = tf.get_variable("b_conv5", [num_filters * 3])
h_conv5 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(dropout2, W_conv5, strides=[1, 1, 1, 1], padding="SAME"), b_conv5)))

# 2D convolution layer
W_conv6 = tf.get_variable("W_conv6", [window_size, window_size, num_filters * 3, num_filters * 3])
b_conv6 = tf.get_variable("b_conv6", [num_filters * 3])
h_conv6 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(h_conv5, W_conv6, strides=[1, 1, 1, 1], padding="SAME"), b_conv6)))

# fractional max pooling layer
h_pool3 = tf.nn.fractional_max_pool(h_conv6, pooling_ratio, pseudo_random=True, overlapping=True)[0]

# dropout layer
dropout3 = tf.nn.dropout(h_pool3, 0.7)

# 2D convolution layer
W_conv7 = tf.get_variable("W_conv7", [window_size, window_size, num_filters * 3, num_filters * 4])
b_conv7 = tf.get_variable("b_conv7", [num_filters * 4])
h_conv7 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(dropout3, W_conv7, strides=[1, 1, 1, 1], padding="SAME"), b_conv7)))

# 2D convolution layer
W_conv8 = tf.get_variable("W_conv8", [window_size, window_size, num_filters * 4, num_filters * 4])
b_conv8 = tf.get_variable("b_conv8", [num_filters * 4])
h_conv8 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(h_conv7, W_conv8, strides=[1, 1, 1, 1], padding="SAME"), b_conv8)))

# fractional max pooling layer
h_pool4 = tf.nn.fractional_max_pool(h_conv8, pooling_ratio, pseudo_random=True, overlapping=True)[0]

# dropout layer
dropout4 = tf.nn.dropout(h_pool4, 0.7)

# 2D convolution layer
W_conv9 = tf.get_variable("W_conv9", [window_size, window_size, num_filters * 4, num_filters * 5])
b_conv9 = tf.get_variable("b_conv9", [num_filters * 5])
h_conv9 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(dropout4, W_conv9, strides=[1, 1, 1, 1], padding="SAME"), b_conv9)))

# 2D convolution layer
W_conv10 = tf.get_variable("W_conv10", [window_size, window_size, num_filters * 5, num_filters * 5])
b_conv10 = tf.get_variable("b_conv10", [num_filters * 5])
h_conv10 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(h_conv9, W_conv10, strides=[1, 1, 1, 1], padding="SAME"), b_conv10)))

# fractional max pooling layer
h_poo5 = tf.nn.fractional_max_pool(h_conv10, pooling_ratio, pseudo_random=True, overlapping=True)[0]

# dropout layer
dropout5 = tf.nn.dropout(h_poo5, 0.6)

# 2D convolution layer
W_conv11 = tf.get_variable("W_conv11", [window_size, window_size, num_filters * 5, num_filters * 6])
b_conv11 = tf.get_variable("b_conv11", [num_filters * 6])
h_conv11 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(dropout5, W_conv11, strides=[1, 1, 1, 1], padding="SAME"), b_conv11)))

# 2D convolution layer
W_conv12 = tf.get_variable("W_conv12", [window_size, window_size, num_filters * 6, num_filters * 6])
b_conv12 = tf.get_variable("b_conv12", [num_filters * 6])
h_conv12 = tf.contrib.layers.batch_norm(tf.nn.relu(tf.add(tf.nn.conv2d(h_conv11, W_conv12, strides=[1, 1, 1, 1], padding="SAME"), b_conv12)))

# fractional max pooling layer
h_pool6 = tf.nn.fractional_max_pool(h_conv12, pooling_ratio, pseudo_random=True, overlapping=True)[0]

# dropout layer
dropout6 = tf.nn.dropout(h_pool6, 0.6)

# reshape layer
dim = dropout6.get_shape()[1].value ** 2 * dropout6.get_shape()[3].value
reshape = tf.reshape(dropout6, [-1, dim])

# output layer
W_conv13 = tf.get_variable("W_conv13", [dim, num_classes])
b_conv13 = tf.get_variable("b_conv13", [num_classes])
output = tf.add(tf.matmul(reshape, W_conv13), b_conv13)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_pred, tf.float32))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(log_save_path,sess.graph)
    total_batch = int(X_train.shape[0] / batch_size)
    total_batch_test = int(X_test.shape[0] / batch_size)

    for i in range(epochs):
        print("epoch %d/%d:" % (i + 1, epochs))
        
        train_loss = 0
        train_acc = 0
        
        for batch in range(total_batch):
            batch_x = X_train[batch * batch_size: (batch + 1) * batch_size, :]
            batch_y = y_train[batch * batch_size: (batch + 1) * batch_size, :]
            
            batch_x = data_augmentation(batch_x, crop_shape, img_size)
            
            _, batch_loss, batch_acc = sess.run([optimizer, loss, accuracy], feed_dict={x: batch_x, y: batch_y})
            
            train_loss += batch_loss
            train_acc += batch_acc
        
            if batch == total_batch - 1:
                train_loss /= total_batch
                train_acc /= total_batch
                train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), tf.Summary.Value(tag="train_accuracy", simple_value=train_acc)])

                test_loss = 0
                test_acc = 0
                for test_batch in range(total_batch_test):
                    batch_x_test = X_test[test_batch * batch_size: (test_batch + 1) * batch_size, :]
                    batch_y_test = y_test[test_batch * batch_size: (test_batch + 1) * batch_size, :]
                    batch_loss_test, batch_acc_test = sess.run([loss, accuracy], feed_dict={x: batch_x_test, y: batch_y_test})
                    
                    test_loss += batch_loss_test
                    test_acc += batch_acc_test

                test_loss /= total_batch_test
                test_acc /= total_batch_test
                test_summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", simple_value=test_loss), tf.Summary.Value(tag="test_accuracy", simple_value=test_acc)])
                
                summary_writer.add_summary(train_summary, i)
                summary_writer.add_summary(test_summary, i)
                summary_writer.flush()
                print("iteration: %d/%d, train_loss: %.4f, train_accuracy: %.4f, test_loss: %.4f, test_accuracy: %.4f" % (batch + 1, total_batch, train_loss, train_acc, test_loss, test_acc))
            elif (batch + 1) % 100 == 0:
                print("iteration: %d/%d, train_loss: %.4f, train_accuracy: %.4f" % (batch + 1, total_batch, train_loss / (batch + 1), train_acc / (batch + 1)))
    save_path = saver.save(sess, model_save_path)
    print("Model saved in file: %s" % save_path)

Data has apparently already been downloaded and unpacked.
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
epoch 1/200:
iteration: 100/500, train_loss: 2.6539, train_accuracy: 22.7300
iteration: 200/500, train_loss: 2.3645, train_accuracy: 26.9050
iteration: 300/500, train_loss: 2.2054, train_accuracy: 30.4333
iteration: 400/500, train_loss: 2.0993, train_accuracy: 32.8100
iteration: 500/500, train_loss: 2.0145, train_accuracy: 34.7400, test_loss: 1.6428, test_accuracy: 44.7400
epoch 2/200:
iteration: 100/500, train_loss: 1.5964, train_accuracy: 44.8300
iteration: 200/500, train_loss: 1.5795, train_accuracy: 46.2050
iteration: 300/500, train_loss: 1.5733, train_accuracy: 46.69

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = "/content/drive/logs/"
get_ipython().system_raw(
    "tensorboard --logdir {} --host 0.0.0.0 --port 6006 &"
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-12-17 05:41:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.226.180.131, 34.206.36.121, 34.206.130.40, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.226.180.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  20.6MB/s    in 0.2s    

2018-12-17 05:41:32 (20.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
http://88b05e51.ngrok.io
